# BirdClef+ 2025 Training Baseline

## Import libraries

In [1]:
import os
import time
import random
import configparser
from pathlib import Path
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold

import torch
import torchaudio
import torchaudio.transforms as at
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

import timm
import wandb

from score.metric import score

import warnings

warnings.filterwarnings('ignore')

## Config

In [2]:
config = configparser.ConfigParser()

In [3]:
config['project'] = {
    'name': 'birdclef_2025',
    'project_path': '/mnt/d/Projects_D/BirdCLEF_2025'
}

config['data'] = {
    'data_path': config['project']['project_path'] + '/data',
    'birdclef_2025': config['project']['project_path'] + '/data/birdclef_2025',
    'processed_audio':config['project']['project_path'] + '/data/audio_processed',
}

config['audio_params'] = {
    'wav_sec': 5,
    'sample_rate': 32000,
}

config['audio_preprocessing'] = {
    'min_segment': 32000 * 5,
    'backend': 'soundfile'
}

config['mel_spectrogram'] = {
    'n_fft': 1024,
    'win_length': 1024,
    'hop_length': 512,
    'n_mels': 128,
    'f_min': 50,
    'f_max': 14000,
    'mel_scale': 'htk',
}

config['model'] = {
    'model_backbone': 'efficientnet',
    'model_desc': 'efficientnet_b0',
    'model_training': 'cv',
    'model_ver': 'v1',
    'model_path': config['project']['project_path'] + '/models'
}

config['notebook_mode'] = {
    'mode': 'train'
}

config['training'] = {
    'processed_audio': config['data']['processed_audio'] + '/v1',
    'device': torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    'epochs': 30,
    'batch_size': 40,
    'num_workers': 8,
    'learning_rate': 5e-4,
    'min_lr': 1e-6,
    'optimizer': 'adamw',
    'scheduler': 'cosineannealinglr',
    'criterion': 'bce',
    'weight_decay': 1e-5,
    't_max': 10,
    'n_folds': 5
}

## WandB

In [4]:
config['project']['project_path'] + '/wandb'

'/mnt/d/Projects_D/BirdCLEF_2025/wandb'

In [5]:
os.path.exists(config['project']['project_path'] + '/wandb')

True

In [6]:
wandb_group = f"{config['model']['model_desc']}_{config['model']['model_training']}-{config['model']['model_ver']}"

In [7]:
wandb_group

'efficientnet_b0_cv-v1'

## Load data

In [11]:
meta = pd.read_csv(config['data']['birdclef_2025'] + '/train.csv')
taxonomy = pd.read_csv(config['data']['birdclef_2025'] + '/taxonomy.csv')

In [12]:
meta.head()

,primary_label,secondary_labels,type,filename,collection,rating,url,latitude,longitude,scientific_name,common_name,author,license
0,1139490,[''],[''],1139490/CSA36385.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
1,1139490,[''],[''],1139490/CSA36389.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3206,-73.7128,Ragoniella pulchella,Ragoniella pulchella,Fabio A. Sarria-S,cc-by-nc-sa 4.0
2,1192948,[''],[''],1192948/CSA36358.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
3,1192948,[''],[''],1192948/CSA36366.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.2800,-73.8582,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0
4,1192948,[''],[''],1192948/CSA36373.ogg,CSA,0.0,http://colecciones.humboldt.org.co/rec/sonidos...,7.3791,-73.7313,Oxyprora surinamensis,Oxyprora surinamensis,Fabio A. Sarria-S,cc-by-nc-sa 4.0


In [13]:
class_labels = meta.primary_label.unique()
num_classes = len(class_labels)

## Utils

In [14]:
def extract_metrics(metrics):
    metrics = {k:v.item() for k, v in metrics.items()}
    return metrics

## Dataset

In [15]:
class BirdclefDataset(Dataset):
    def __init__(self, df, class_labels, mode='train'):
        self.df = df
        self.class_labels = class_labels
        self.mode = mode
        self.config = config
        self.input_path = Path(config['data']['processed_audio']) / 'v1'
        self.min_segment = int(config['audio_preprocessing']['min_segment'])
        self.mode = config['notebook_mode']['mode']

    @classmethod
    def normalize_std(cls, spec, eps=1e-23):
        mean = torch.mean(spec)
        std = torch.std(spec)
        return (spec - mean) / (std + eps)

    def get_mel_spectrogram(self, audio_signal):
        params = self.config['mel_spectrogram']
        mel_spectrogram = at.MelSpectrogram(
            sample_rate=int(self.config['audio_params']['sample_rate']),
            n_fft=int(params['n_fft']),
            win_length=int(params['win_length']),
            hop_length=int(params['hop_length']),
            n_mels=int(params['n_mels']),
            f_min=float(params['f_min']),
            f_max=float(params['f_max']),
            mel_scale=params['mel_scale']
        )
        mel_spec = mel_spectrogram(audio_signal)
        return torch.log(mel_spec)

    def __getitem__(self, index):
        filename = self.df.iloc[index].filename
        sig, _ = torchaudio.load(self.input_path / filename, backend=self.config['audio_preprocessing']['backend'])
        sig = sig / torch.max(torch.abs(sig))
        sig = sig + 1.5849e-05 * (torch.rand(1, self.min_segment) - 0.5)

        mel_spec = self.get_mel_spectrogram(sig)
        mel_spec = self.normalize_std(mel_spec)

        if self.mode == 'train' and random.random() < 0.5:
            mel_spec = self.apply_augmentation(mel_spec)

        target = self.df.iloc[index].primary_label
        y = np.array([1 if lbl == target else 0 for lbl in self.class_labels])

        return mel_spec, torch.tensor(y, dtype=torch.float32)

    @classmethod
    def apply_augmentation(cls, mel_spec):

        # Time masking (horizontal stripes)
        if random.random() < 0.5:
            num_masks = random.randint(1, 3)
            for _ in range(num_masks):
                width = random.randint(5, 20)
                start = random.randint(0, mel_spec.shape[2] - width)
                mel_spec[0, :, start:start+width] = 0

        # Frequency masking (vertical stripes)
        if random.random() < 0.5:
            num_masks = random.randint(1, 3)
            for _ in range(num_masks):
                height = random.randint(5, 20)
                start = random.randint(0, mel_spec.shape[1] - height)
                mel_spec[0, start:start+height, :] = 0

        # Random brightness/contrast
        if random.random() < 0.5:
            gain = random.uniform(0.8, 1.2)
            bias = random.uniform(-0.1, 0.1)
            mel_spec = mel_spec * gain + bias
            mel_spec = torch.clamp(mel_spec, 0, 1)

        return mel_spec

    def __len__(self):
        return len(self.df)

## Model

In [16]:
class BirdclefModel(pl.LightningModule):
    def __init__(self, class_labels):
        super(BirdclefModel, self).__init__()
        self.save_hyperparameters()
        self.class_labels = class_labels
        self.num_classes = len(self.class_labels)
        self.model_backbone = config['model']['model_backbone']
        self.model = self.get_model()

        self.hparams.learning_rate = float( config['training']['learning_rate'])
        self.hparams.weight_decay = float(config['training']['weight_decay'])
        self.hparams.min_lr = float(config['training']['min_lr'])
        self.hparams.t_max = int(config['training']['t_max'])
        
        self.optimizer = config['training']['optimizer']
        self.scheduler = config['training']['scheduler']
        self.criteria = config['training']['criterion']

        self.loss_fn = self.get_criteria()

        # For storing predictions and labels
        self.pred_train = []
        self.label_train = []
        self.pred_val = []
        self.label_val = []

    def get_model(self):

        model_obj = timm.create_model(config['model']['model_desc'], pretrained=True, num_classes=self.num_classes)
        return model_obj

    def forward(self, x):
        x = torch.cat((x, x, x), 1)
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.loss_fn(preds, y)

        self.label_train.append(y.detach().cpu().numpy())
        self.pred_train.append(torch.softmax(preds, dim=1).detach().cpu().numpy())

        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        preds = self(x)
        loss = self.loss_fn(preds, y)

        self.label_val.append(y.detach().cpu().numpy())
        self.pred_val.append(torch.softmax(preds, dim=1).detach().cpu().numpy())

        self.log("val_loss", loss, on_epoch=True)
        return loss

    def on_train_epoch_end(self):
        auc = self.cal_score(self.label_train, self.pred_train)
        self.log("train_auc", auc, on_epoch=True)
        self.pred_train.clear()
        self.label_train.clear()

    def on_validation_epoch_end(self):
        auc = self.cal_score(self.label_val, self.pred_val)
        self.log("val_auc", auc, on_epoch=True)
        self.pred_val.clear()
        self.label_val.clear()

    def configure_optimizers(self):
        optimizer = self.get_optimizer()
        scheduler = self.get_scheduler(optimizer)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss"
            }
        }

    def get_optimizer(self):
        if self.optimizer == 'adam':
            return torch.optim.Adam(self.parameters(),
                                    lr=self.hparams.learning_rate,
                                    weight_decay=self.hparams.weight_decay)
        elif self.optimizer == 'adamw':
            return torch.optim.AdamW(self.parameters(),
                                     lr=self.hparams.learning_rate,
                                     weight_decay=self.hparams.weight_decay)
        elif self.optimizer == 'sgd':
            return torch.optim.SGD(self.parameters(),
                                   lr=self.hparams.learning_rate,
                                   momentum=0.9,
                                   weight_decay=self.hparams.weight_decay)
        else:
            raise NotImplementedError

    def get_scheduler(self, optimizer):
        if self.scheduler == 'reducelronplateau':
            return torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                              mode='min',
                                                              factor=0.5,
                                                              patience=2,
                                                              min_lr=self.hparams.min_lr)
        elif self.scheduler == 'steplr':
            return torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.5)
        elif self.scheduler == 'cosineannealinglr':
            return torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                              T_max=self.hparams.t_max,
                                                              eta_min=self.hparams.min_lr)
        else:
            return None

    def get_criteria(self):
        if self.criteria == 'bce':
            return nn.BCEWithLogitsLoss()
        elif self.criteria == 'ce':
            return nn.CrossEntropyLoss()
        else:
            raise NotImplementedError

    def cal_score(self, label, pred):
        label = np.concatenate(label)
        pred = np.concatenate(pred)

        label_df = pd.DataFrame(label>0.5, columns=self.class_labels)
        pred_df = pd.DataFrame(pred, columns=self.class_labels)
        label_df['id'] = np.arange(len(label_df))
        pred_df['id'] = np.arange(len(pred_df))

        return score(label_df, pred_df, row_id_column_name='id')

## Training

In [17]:
skf = StratifiedKFold(n_splits=int(config['training']['n_folds']), shuffle=True, random_state=42)

In [18]:
fold_results = []

for fold, (train_index, val_index) in enumerate(skf.split(meta, meta.primary_label)):
    print(f"Training fold {fold + 1}")

    wandb.init(project=config['project']['name'],
               name=f"{config['model']['model_desc']}_{config['model']['model_training']}-fold_{fold}-{config['model']['model_ver']}",
               dir=config['project']['project_path'],
               group=wandb_group)

    train_df = meta.iloc[train_index]
    val_df = meta.iloc[val_index]

    train_dataset = BirdclefDataset(train_df, mode='train', class_labels=class_labels)
    train_loader = DataLoader(train_dataset, batch_size=int(config['training']['batch_size']), shuffle=True, num_workers=int(config['training']['num_workers']),drop_last=True)

    val_dataset = BirdclefDataset(val_df, mode='test', class_labels=class_labels)
    val_loader = DataLoader(val_dataset, batch_size=int(config['training']['batch_size']), shuffle=False, num_workers=int(config['training']['num_workers']),drop_last=True)

    model = BirdclefModel(class_labels)

    checkpoint_callback = ModelCheckpoint(
        dirpath=config['model']['model_path'] + '/'+ config['model']['model_desc'] + '_' + config['model']['model_training'] + '-' + config['model']['model_ver'],
        filename=f'{fold}-' + '{epoch:02d}-{val_loss:.6f}',
        save_top_k=1,
        monitor='val_loss',
        mode='min'
    )

    early_stopping = EarlyStopping(monitor="val_loss", patience=4, mode="min")

    logger = WandbLogger(name=f"fold_{fold}", log_model='all')

    trainer = pl.Trainer(
        max_epochs=int(config['training']['epochs']),
        devices=1,
        accelerator='gpu' if torch.cuda.is_available() else 'cpu',
        logger=logger,
        callbacks=[checkpoint_callback, early_stopping]
    )

    trainer.fit(model, train_loader, val_loader)

    metrics = extract_metrics(trainer.callback_metrics)

    fold_results.append({
        'fold': fold,
        'metrics': metrics
    })

    wandb.finish()

    print(f"Fold {fold} completed")

Training fold 1


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: siddhantmahalle (siddhant-ai) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3070 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params | Mode 
------------------------------------------------------
0 | model   | EfficientNet      | 4.3 M  | train
1 | loss_fn | BCEWithLogitsLoss | 0      | train
------------------------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.086    Total estimated model params size (MB)
338       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇███
train_auc,▁▆▇▇█████████
train_loss_epoch,█▄▃▃▂▂▂▁▁▁▁▁▁
train_loss_step,██▇▇▇▆▅▆▅▄▄▄▃▃▄▄▃▄▃▃▂▂▂▁▂▁▂▂▂▁▁▃▁▁▂▂▂▁▁▁
trainer/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇██
val_auc,▁▅▇██████████
val_loss,█▅▃▂▁▁▁▁▁▁▁▁▁
epoch,12
train_auc,0.9984
train_loss_epoch,0.00388
train_loss_step,0.00272


Fold 4 completed


In [19]:
fold_results

[{'fold': 0,
  'metrics': {'train_loss': 0.004395403899252415,
   'train_loss_step': 0.004206269513815641,
   'val_loss': 0.012511368840932846,
   'val_auc': 0.962357759475708,
   'train_loss_epoch': 0.004395403899252415,
   'train_auc': 0.9981595873832703}},
 {'fold': 1,
  'metrics': {'train_loss': 0.003985675051808357,
   'train_loss_step': 0.0031179357320070267,
   'val_loss': 0.010683638975024223,
   'val_auc': 0.9681699275970459,
   'train_loss_epoch': 0.003985675051808357,
   'train_auc': 0.9983196258544922}},
 {'fold': 2,
  'metrics': {'train_loss': 0.00413606408983469,
   'train_loss_step': 0.0033246721141040325,
   'val_loss': 0.011996230110526085,
   'val_auc': 0.9674593210220337,
   'train_loss_epoch': 0.00413606408983469,
   'train_auc': 0.9982581734657288}},
 {'fold': 3,
  'metrics': {'train_loss': 0.003917776979506016,
   'train_loss_step': 0.002337726065889001,
   'val_loss': 0.011340056546032429,
   'val_auc': 0.9699891805648804,
   'train_loss_epoch': 0.003917776979506